<div align="center">
    <h1><b><u>TRAFFIC OPTIMIZATION</u></b></h1>
</div>


**GROUP NUMBER: 7**
     
**GROUP MEMBERS**
   
- **Adebola**
- **Rahim**
- **Sayeed**
- **Yinka**
- **Minto**
   

<div align="center">
    <h3><b><u>7. Random Forest</u></b></h3>
</div>


### 1. Data Load

In [17]:
# Import Libraries 
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import joblib


In [19]:
# Load your dataset
data = pd.read_csv('data/cleaned_data.csv')

In [20]:
data.head()

,ROAD_CLASS,DISTRICT,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,DIVISION
0,Major Arterial,Toronto and East York,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,50 to 54,Major,East,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,D55
1,Major Arterial,Toronto and East York,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,15 to 19,Minor,East,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,D55
2,Major Arterial,Toronto and East York,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Driver,55 to 59,Minor,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,D55
3,Major Arterial,Toronto and East York,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,20 to 24,Minor,East,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,D55
4,Major Arterial,Toronto and East York,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,15 to 19,Minor,East,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,D55


### 2. Data Pre-processing

In [21]:
# Encode target
injury_mapping = {"Minimal": 0, "Minor": 1, "Major": 2, "Fatal": 3}
data["INJURY"] = data["INJURY"].map(injury_mapping)

In [22]:
# Feature and target definition
features = [
    "DRIVCOND", "INVTYPE", "IMPACTYPE", "ACCLASS"
]
X = data[features]
y = data["INJURY"]

In [23]:
# Define preprocessing
preprocessor = ColumnTransformer(transformers=[
    
    ("cat", OneHotEncoder(handle_unknown="ignore"), [
        "DRIVCOND", "INVTYPE", "IMPACTYPE", "ACCLASS"
    ])
])

In [24]:
# Create and train pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=50,
    max_depth=10,
    min_samples_split=5, class_weight='balanced'))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['DRIVCOND', 'INVTYPE',
                                                   'IMPACTYPE', 'ACCLASS'])])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=10,
                                        min_samples_split=5,
                                        n_estimators=50))])

In [25]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.6521624472573839
              precision    recall  f1-score   support

           0       0.14      0.62      0.22       229
           1       0.23      0.21      0.22       307
           2       0.96      0.68      0.79      3073
           3       0.63      0.98      0.77       183

    accuracy                           0.65      3792
   macro avg       0.49      0.62      0.50      3792
weighted avg       0.83      0.65      0.71      3792



In [11]:
# Save the pipeline
#joblib.dump(pipeline, "rf_pipeline_raw_input.pkl")

In [26]:
import pickle

# Save the trained pipeline
with open("rf_pipeline_raw_input.pkl", "wb") as f:
    pickle.dump(pipeline, f)